<a href="https://colab.research.google.com/github/skywalker00001/Conterfactual-Reasoning-Project/blob/main/data_cleaning_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# how to find out the different or changed blocks in sentence b compared to the sentence a (base)

In [75]:
# Importing stock libraries
import numpy as np
import pandas as pd
import difflib
import nltk
import copy
import regex as re

In [57]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [58]:

# a = "I paid the cashier and patiently waited for my drink."
# b = "I paid the cashier and patiently waited at the counter for my drink."
a = "I want to acchieve something to school."
b = "I do something to go for it to school."
a_list = nltk.word_tokenize(a)
b_list = nltk.word_tokenize(b)

a_list = "qabxcd"
b_list = "abycdf"
print(a_list)
print(b_list)

qabxcd
abycdf


In [59]:
s = difflib.SequenceMatcher(None, a_list, b_list)
for block in s.get_matching_blocks():
    print(block)
    

Match(a=1, b=0, size=2)
Match(a=4, b=3, size=2)
Match(a=6, b=6, size=0)


In [60]:
print(a_list[2])

b


In [61]:
print(a_list[0:0+8])
print(a_list[8:8+3])
print(a_list[11:11+0])

qabxcd




In [62]:
matches = []
matches.append([0, 0, 0])
for block in s.get_matching_blocks():
    #matches.append([block[0], block[1], block[2]])
    matches.append([i for i in block])
    #matches.append(block)
print(matches)   

# explanation: matches[i][0] are the a index, matches[i][1] are the b index, matches[i] [2] are the lengths of same (matched) words.

[[0, 0, 0], [1, 0, 2], [4, 3, 2], [6, 6, 0]]


In [63]:
changes = []
for i in range(len(matches) - 1):
    #print(matches[i])
    if ((matches[i][0]+ matches[i][2] < matches[i+1][0]) & (matches[i][1]+ matches[i][2] < matches[i+1][1])):     # replace
        string = (" ").join(b_list[(matches[i][1]+matches[i][2]) : matches[i+1][1]])
        changes.append(f"replacing_{matches[i][0]+matches[i][2]}-{matches[i+1][0]}:\"{string}\".")
    elif ((matches[i][0]+ matches[i][2] < matches[i+1][0]) & (matches[i][1]+ matches[i][2] == matches[i+1][1])):     # delete
        string = (" ").join(b_list[(matches[i][1]+matches[i][2]) : matches[i+1][1]])
        changes.append(f"deleting_{matches[i][0]+matches[i][2]}-{matches[i+1][0]}:\"{string}\".")
    elif ((matches[i][0]+ matches[i][2] == matches[i+1][0]) & (matches[i][1]+ matches[i][2] < matches[i+1][1])):     # insert
        string = (" ").join(b_list[(matches[i][1]+matches[i][2]) : matches[i+1][1]])
        changes.append(f"inserting_{matches[i][0]+matches[i][2]}-{matches[i+1][0]}:\"{string}\".")

In [64]:
print(changes)

['deleting_0-1:"".', 'replacing_3-4:"y".', 'inserting_6-6:"f".']


In [48]:
c = copy.deepcopy(a_list)
c[7:7] = nltk.word_tokenize("at the counter")


TypeError: ignored

In [37]:
print(a_list)

['I', 'want', 'to', 'acchieve', 'something', 'to', 'school', '.']


In [38]:
print(c)

['I', 'want', 'to', 'acchieve', 'something', 'to', 'school', 'at', 'the', 'counter', '.']


# another way

In [69]:
# a = "qabxcd"
# b = "abycdf"
cgs = []
s = difflib.SequenceMatcher(None, a_list, b_list)
for tag, i1, i2, j1, j2 in s.get_opcodes():
    print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}'.format(tag, i1, i2, j1, j2, a_list[i1:i2], b_list[j1:j2]))
    #print('{:7}   a[{}:{}] --> b[{}:{}] {!r} --> {!r}'.format(tag, i1, i2, j1, j2, a[i1:i2], b[j1:j2]))
    if (tag != 'equal'):
        #cgs.append(f'{tag}_{i1}-{i2}:\"{(" ").join(b_list[j1:j2])}\".')
        cgs.append(f{i1}-{i2}:\"{(" ").join(b_list[j1:j2])}\".')

delete    a[0:1] --> b[0:0]      'q' --> ''
equal     a[1:3] --> b[0:2]     'ab' --> 'ab'
replace   a[3:4] --> b[2:3]      'x' --> 'y'
equal     a[4:6] --> b[3:5]     'cd' --> 'cd'
insert    a[6:6] --> b[5:6]       '' --> 'f'


In [70]:
print(cgs)

['0-1:"".', '3-4:"y".', '6-6:"f".']


In [71]:
t=(" ".join(cgs))
print(t)

0-1:"". 3-4:"y". 6-6:"f".


In [72]:
print(t.split(' '))

['0-1:"".', '3-4:"y".', '6-6:"f".']


function

In [129]:
def origin2edited(origin, edited):
    origin_list = nltk.word_tokenize(origin)
    editeed_list = nltk.word_tokenize(edited)
    s = difflib.SequenceMatcher(None, origin_list, editeed_list)
    cgs = []
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if (tag != 'equal'):
            cgs.append(f'{i1}-{i2}:\"{(" ").join(editeed_list[j1:j2])}\".')
    conclusion = "".join(cgs)
    return conclusion

In [133]:
print(origin2edited(a, b))


1-2:"do something".3-5:"go for it".


In [140]:
cgs = sen.split('.')[0:-1]
print(len(cgs))

2


In [154]:
def restore(cg, origin):
    cgs = cg.split('.')[0:-1]
    origin_list = nltk.word_tokenize(origin)
    #blocks = copy.deepcopy(origin)
    for j in list(reversed(cgs)):
        #print("unchanged: ", origin_list)
        #print(j)
        pattern = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')  # matches 1-2:"do something".  (1), (2), (do something)
        results = re.search(pattern, j)  # i3 means the content between ""
        i1, i2, i3 = results.group(1), results.group(2), results.group(3)
        origin_list[int(i1): int(i2)] = nltk.word_tokenize(i3)
        #print("changed: ", origin_list)
    return (" ").join(origin_list[0:-1])+'.'  #because we don't want any space before '.', which is the last element in the origin_list

In [144]:
print(a)

I want to acchieve something to school.


In [145]:
print(sen)

1-2:"do something".3-5:"go for it".


In [146]:
b = "I do something to go for it to school."

In [153]:
print(restore(origin2edited(a, b), a))

unchanged:  ['I', 'want', 'to', 'acchieve', 'something', 'to', 'school', '.']
3-5:"go for it"
changed:  ['I', 'want', 'to', 'go', 'for', 'it', 'to', 'school', '.']
unchanged:  ['I', 'want', 'to', 'go', 'for', 'it', 'to', 'school', '.']
1-2:"do something"
changed:  ['I', 'do', 'something', 'to', 'go', 'for', 'it', 'to', 'school', '.']
I do something to go for it to school.


In [134]:
i1 = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')
#i1 = re.compile(r'(\d+)')
sen = "1-2:\"do something\".3-5:\"go for it\"."
i3 = re.search(i1, sen).group(1)
print(i3)

1


In [104]:
pattern = re.compile(r'^(\d+)-(\d+):\"(.*?)\"')  # matches 1-2:"do something".  (1), (2), (do something)
results = re.search(pattern, sen)  # i3 means the content between ""
i1, i2, i3 = results.group(1), results.group(2), results.group(3)
print(i1, i2, i3)

1 2 do something


In [106]:
print(restore(orgin2edited(a, b), a))

TypeError: ignored